In [1]:
#init libs
import math
import csv
import numpy as np
from numpy import loadtxt
from tensorflow.python.keras import  Model
from tensorflow.python.keras.utils import plot_model
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import model_from_json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K

In [2]:
#init datasets
#403-19
csv_train=open('processed_data/train_dataset.txt', newline='')
reader_train = csv.reader(csv_train, delimiter=',', quotechar='|')

csv_test=open('processed_data/test_dataset.txt', newline='')
reader_test = csv.reader(csv_test, delimiter=',', quotechar='|')

X_train_dataset=[]
X_test_dataset=[]

Y_train_dataset=[]
Y_test_dataset=[]



In [3]:
#control variables
target_size=10#num of position for elements
num_elements=17# params of element in formula



In [4]:
#preprocessing datasets

train_num_fromulas=0
test_num_fromulas=0
counter_tr=0    
read_it=True

while(read_it==True):
    try:
        counter_tr=counter_tr+1
        read_row = next(reader_train)
        crit_temp=read_row[-1]
        Y_train_dataset.append(float(crit_temp))
        
        buf=[]
        for i in range(len(read_row)-1):
            c=read_row[i].split(" ")
            for j in range(len(c)):
                c[j]=float(c[j])
            c=np.asarray(c)
            buf.append(c)
        buf=np.asarray(buf)
        X_train_dataset.append(buf)
        train_num_fromulas=train_num_fromulas+1
    except StopIteration:
        read_it=False
csv_train.close() 

counter_ts=0

read_it=True

while(read_it==True):
    try:
        counter_ts=counter_ts+1
        read_row = next(reader_test)
        crit_temp=read_row[-1]
        Y_test_dataset.append(float(crit_temp))
    
        buf=[]
        for i in range(len(read_row)-1):
            c=read_row[i].split(" ")
            for j in range(len(c)):
                c[j]=float(c[j])
            c=np.asarray(c)
            buf.append(c)
        buf=np.asarray(buf)
        
        X_test_dataset.append(buf)
        test_num_fromulas=test_num_fromulas+1
    except StopIteration:
        read_it=False
csv_test.close() 


X_test_dataset=np.asarray(X_test_dataset)
X_train_dataset=np.asarray(X_train_dataset)

#zero buffer
add_arr=[]
for i in range(num_elements):
    add_arr.append(0)
add_arr=np.asarray(add_arr)

X_test_dataset_processed=[]
Y_test_dataset_processed=[]

X_val_dataset_processed=[]
Y_val_dataset_processed=[]

X_train_dataset_processed=[]
Y_train_dataset_processed=[]


for i in range(X_test_dataset.shape[0]):

    c=X_test_dataset[i].shape[0]
    dist=target_size-X_test_dataset[i].shape[0]
    buf1=[]
    
    if dist>0:
        for j in range(dist):
            buf1.append(add_arr)
    for j in range(X_test_dataset[i].shape[0]):
        buf1.append(X_test_dataset[i][j])
    buf1=np.asarray(buf1)
    X_test_dataset_processed.append(buf1)
    Y_test_dataset_processed.append(Y_test_dataset[i])

for i in range(X_train_dataset.shape[0]):

    c=X_train_dataset[i].shape[0]
    dist=target_size-X_train_dataset[i].shape[0]
    buf1=[]
    
    if dist>0:
        for j in range(dist):
            buf1.append(add_arr)
    for j in range(X_train_dataset[i].shape[0]):
        buf1.append(X_train_dataset[i][j])
    buf1=np.asarray(buf1)
    X_train_dataset_processed.append(buf1)
    Y_train_dataset_processed.append(Y_train_dataset[i])


X_test_dataset_processed=np.asarray(X_test_dataset_processed)
X_train_dataset_processed=np.asarray(X_train_dataset_processed)


95680
10635


In [ ]:


#R2 loss function
def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


loss='mse'#mae or mse or R2. RMSE is square root by MSE
#loss=coeff_determination

optimiser=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

metric='mae'# mae or mse or R2

#loading model
model_json="pretrained_nn/best_mix.json"
model_h5="pretrained_nn/best_mix.h5"

json_file = open(model_json, "r")
loaded_model_json = json_file.read()
json_file.close()


model_loaded = model_from_json(loaded_model_json)
model_loaded.load_weights(model_h5)
print("Done")
model = Model(model_loaded.input, model_loaded.layers[-1].output)

model.summary()
model.compile(optimizer=optimiser, loss=loss, metrics=[metric])
plot_model(model, show_shapes=True, expand_nested=True)



In [ ]:
#Different GPU can give different results. MAE can have some changes between 3.99 and 4.18 .
#Results from article was getting by Google Colab on GPU Testla K80. There MAE will be 4.079 (tested in February 2020)
mse, mae = model.evaluate(X_test_dataset_processed, Y_test_dataset_processed, verbose=2)
print("mae mistake:", mae)
print("mistake mse:", mse)


In [11]:
#you can test it for different temperatures

#temperature diapason
min_val=90#minimum temperature
max_val=999#maximum temperature

X_test_0=[]
Y_test_0=[]
print("test")
for i in range(len(Y_test_dataset_processed)):
    if Y_test_dataset_processed[i]>min_val and Y_test_dataset_processed[i]<max_val:
        X_test_0.append(X_test_dataset_processed[i])
        Y_test_0.append(Y_test_dataset_processed[i])
X_test_0=np.asarray(X_test_0)
pred = model.predict(X_test_0)

mistake=0
a=[0,0,0,0,0]
for i in range(len(Y_test_0)):
    mistake=mistake+abs(pred[i][0]-Y_test_0[i])
    if abs(pred[i][0]-Y_test_0[i])<1:
        a[0]=a[0]+1
    if abs(pred[i][0]-Y_test_0[i])>1 and abs(pred[i][0]-Y_test_0[i])<5:
        a[1]=a[1]+1
    if abs(pred[i][0]-Y_test_0[i])>5 and abs(pred[i][0]-Y_test_0[i])<10:
        a[2]=a[2]+1
    if abs(pred[i][0]-Y_test_0[i])>10 and abs(pred[i][0]-Y_test_0[i])<100:
        a[3]=a[3]+1
mistake=mistake/len(Y_test_0)
print("MAE for these temperatures")
print(mistake)

print((a[0]+a[1]+a[2])/((a[0]+a[1]+a[2]+a[3])))
print("percent of error smaller than 1 degree")
print((a[0])/((a[0]+a[1]+a[2]+a[3])))
print("percent of error smaller than 5 degree")
print((a[1])/((a[0]+a[1]+a[2]+a[3])))
print("percent of error smaller than 10 degree")
print((a[2])/((a[0]+a[1]+a[2]+a[3])))

print("show all results")
for i in range(len(Y_test_0)):
    print("preficted="+str(pred[i][0])+" real="+str(Y_test_0[i]))
X_test_0=[]
Y_test_0=[]




test
MAE for these temperatures
7.053140905303077
0.7976878612716763
percent of error smaller than 1 degree
0.14450867052023122
percent of error smaller than 5 degree
0.3988439306358382
percent of error smaller than 10 degree
0.2543352601156069
show all results
preficted=113.00254 real=108.0
preficted=87.274864 real=92.5
preficted=87.71569 real=96.5
preficted=89.76347 real=92.7
preficted=86.62753 real=91.6
preficted=92.946495 real=130.0
preficted=115.91266 real=126.0
preficted=82.812645 real=91.7
preficted=111.36064 real=112.0
preficted=109.352715 real=100.0
preficted=107.110466 real=106.0
preficted=127.23445 real=120.0
preficted=86.22122 real=107.0
preficted=95.47226 real=92.6
preficted=116.81159 real=123.4
preficted=89.16092 real=98.45
preficted=90.19691 real=90.2
preficted=89.60541 real=94.5
preficted=88.08241 real=90.3
preficted=106.24813 real=106.6
preficted=87.62989 real=90.5
preficted=86.62753 real=91.15
preficted=87.49101 real=97.0
preficted=122.68098 real=125.0
preficted=83.49